In [1]:
import requests
from bs4 import BeautifulSoup
from io import StringIO

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc


c:\Users\mistr\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
constructor_results = pd.read_csv('constructor_results.csv')
constructor_standings = pd.read_csv('constructor_standings.csv')
constructors = pd.read_csv('constructors.csv')
driver_standings = pd.read_csv('driver_standings.csv')
drivers = pd.read_csv('drivers.csv')
lap_times = pd.read_csv('lap_times.csv')
pit_stops = pd.read_csv('pit_stops.csv')
qualifying = pd.read_csv('qualifying.csv')
races = pd.read_csv('races.csv')
results = pd.read_csv('results.csv')
circuits = pd.read_csv('circuits.csv')
seasons = pd.read_csv('seasons.csv')
sprint_results = pd.read_csv('sprint_results.csv')
status = pd.read_csv('status.csv')

In [ ]:
# Step 1: Filter to last 10 years
recent_races = races[races['year'] >= (races['year'].max() - 10)]

# Step 2: Merge datasets - adjusting for the presence of 'raceId' in each dataset
merged_data = results.copy()

# Merge with recent races (this includes 'raceId' and 'circuitId')
if 'raceId' in recent_races.columns:
    merged_data = merged_data.merge(recent_races[['raceId', 'year', 'circuitId']], on='raceId', how='left')

# Merge with circuits dataset
if 'circuitId' in circuits.columns:
    merged_data = merged_data.merge(circuits[['circuitId', 'name']], on='circuitId', how='left')

# Merge with drivers dataset (assuming driverId is in both)
if 'driverId' in drivers.columns:
    merged_data = merged_data.merge(drivers[['driverId', 'Name', 'code']], on='driverId', how='left')

# Merge with constructors dataset (assuming constructorId is in both)
if 'constructorId' in constructors.columns:
    merged_data = merged_data.merge(constructors[['constructorId', 'name']], on='constructorId', how='left')

# Merge with driver_standings dataset
if {'driverId', 'raceId'}.issubset(driver_standings.columns):
    merged_data = merged_data.merge(driver_standings[['driverId', 'raceId', 'points', 'position']], 
                                    on=['driverId', 'raceId'], 
                                    how='left', suffixes=('_driver', '_standings'))

# Merge with constructor_standings dataset
if {'constructorId', 'raceId'}.issubset(constructor_standings.columns):
    merged_data = merged_data.merge(constructor_standings[['constructorId', 'raceId', 'points', 'position']], 
                                    on=['constructorId', 'raceId'], 
                                    how='left', suffixes=('_constructor', '_constructor_standings'))

# Merge with qualifying dataset
if {'raceId', 'driverId'}.issubset(qualifying.columns):
    merged_data = merged_data.merge(qualifying[['raceId', 'driverId', 'position']], 
                                    on=['raceId', 'driverId'], 
                                    how='left', suffixes=('', '_qualifying'))

# Step 3: Add target variable (podium: 1 if positionOrder <= 3, else 0)
merged_data['podium'] = (merged_data['positionOrder'] <= 3).astype(int)

merged_data.to_excel("merged_data.xlsx", index=False)

Merged Data Sample:
   resultId  raceId  driverId  constructorId number  grid position_driver  \
0         1      18         1              1     22     1               1   
1         2      18         2              2      3     5               2   
2         3      18         3              3      7     7               3   
3         4      18         4              4      5    11               4   
4         5      18         5              1     23     3               5   

  positionText  positionOrder  points_driver  ...  name_x               Name  \
0            1              1           10.0  ...     NaN     Lewis Hamilton   
1            2              2            8.0  ...     NaN      Nick Heidfeld   
2            3              3            6.0  ...     NaN       Nico Rosberg   
3            4              4            5.0  ...     NaN    Fernando Alonso   
4            5              5            4.0  ...     NaN  Heikki Kovalainen   

  code      name_y points_standings 

In [4]:
# Display all column names in the merged_data DataFrame
print("Column names in merged_data:")
print(merged_data.columns.tolist())


Column names in merged_data:
['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid', 'position_driver', 'positionText', 'positionOrder', 'points_driver', 'laps', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId', 'year', 'circuitId', 'name_x', 'Name', 'code', 'name_y', 'points_standings', 'position_standings', 'points', 'position', 'position_qualifying', 'podium']


Feature Engineering

In [5]:
# Constructor Experience (By races, +1 for each driver)

# Step 1: Calculate the number of races each constructor has participated in
# Sort by constructorId and raceId to ensure cumulative experience
merged_data = merged_data.sort_values(by=['constructorId', 'raceId'])

# Step 2: Create a constructor experience column, cumulative count of races for each constructor
merged_data['constructor_experience'] = merged_data.groupby('constructorId').cumcount() + 1

# Display the first few rows to verify
print(merged_data[['constructorId', 'raceId', 'constructor_experience']].head(10))


      constructorId  raceId  constructor_experience
7571              1       1                       1
7572              1       1                       2
7579              1       2                       3
7592              1       2                       4
7597              1       3                       5
7598              1       3                       6
7616              1       4                       7
7624              1       4                       8
7641              1       5                       9
7648              1       5                      10


In [6]:
# Driver Experience (By races, +1 for each race)

# Step 1: Sort by driverId and raceId to ensure cumulative experience is calculated sequentially
merged_data = merged_data.sort_values(by=['driverId', 'raceId'])

# Step 2: Create a driver experience column, cumulative count of races for each driver
merged_data['driver_experience'] = merged_data.groupby('driverId').cumcount() + 1

# Display the first few rows to verify
print(merged_data[['driverId', 'raceId', 'driver_experience']].head(10))


      driverId  raceId  driver_experience
7572         1       1                  1
7579         1       2                  2
7598         1       3                  3
7616         1       4                  4
7641         1       5                  5
7664         1       6                  6
7685         1       7                  7
7708         1       8                  8
7730         1       9                  9
7733         1      10                 10


In [ ]:
# Driver Age

# Step 1: Ensure dob is in merged_data by merging with the drivers dataset if necessary
if 'dob' not in merged_data.columns:
    merged_data = merged_data.merge(drivers[['driverId', 'dob']], on='driverId', how='left')

# Step 2: Ensure race date is in merged_data by merging with the races dataset if necessary
if 'date' not in merged_data.columns:
    merged_data = merged_data.merge(races[['raceId', 'date']], on='raceId', how='left')

# Step 3: Convert 'dob' and 'date' to datetime format if they are not already
merged_data['dob'] = pd.to_datetime(merged_data['dob'])
merged_data['date'] = pd.to_datetime(merged_data['date'])

# Step 4: Calculate driver age at the time of each race in years
merged_data['driver_age'] = (merged_data['date'] - merged_data['dob']).dt.days / 365.25

# Display the first few rows to verify
print(merged_data[['driverId', 'raceId', 'dob', 'date', 'driver_age']].head(10))


   driverId  raceId        dob       date  driver_age
0         1       1 1985-01-07 2009-03-29   24.221766
1         1       2 1985-01-07 2009-04-05   24.240931
2         1       3 1985-01-07 2009-04-19   24.279261
3         1       4 1985-01-07 2009-04-26   24.298426
4         1       5 1985-01-07 2009-05-10   24.336756
5         1       6 1985-01-07 2009-05-24   24.375086
6         1       7 1985-01-07 2009-06-07   24.413415
7         1       8 1985-01-07 2009-06-21   24.451745
8         1       9 1985-01-07 2009-07-12   24.509240
9         1      10 1985-01-07 2009-07-26   24.547570


In [8]:
# Driver Wins
# Step 1: Create a binary column indicating if the driver won the race (positionOrder == 1)
merged_data['win'] = (merged_data['positionOrder'] == 1).astype(int)

# Step 2: Calculate cumulative wins for each driver
merged_data = merged_data.sort_values(by=['driverId', 'raceId'])
merged_data['driver_wins'] = merged_data.groupby('driverId')['win'].cumsum()

# Drop the temporary 'win' column if you no longer need it
merged_data.drop(columns='win', inplace=True)

# Display the first few rows to verify
print(merged_data[['driverId', 'raceId', 'positionOrder', 'driver_wins']].head(10))



   driverId  raceId  positionOrder  driver_wins
0         1       1             20            0
1         1       2              7            0
2         1       3              6            0
3         1       4              4            0
4         1       5              9            0
5         1       6             12            0
6         1       7             13            0
7         1       8             16            0
8         1       9             18            0
9         1      10              1            1


In [9]:
# Constructor Wins

# Step 1: Create a binary column indicating if the constructor won the race (positionOrder == 1)
merged_data['constructor_win'] = (merged_data['positionOrder'] == 1).astype(int)

# Step 2: Calculate cumulative wins for each constructor
merged_data = merged_data.sort_values(by=['constructorId', 'raceId'])
merged_data['constructor_wins'] = merged_data.groupby('constructorId')['constructor_win'].cumsum()

# Drop the temporary 'constructor_win' column if you no longer need it
merged_data.drop(columns='constructor_win', inplace=True)

# Display the first few rows to verify
print(merged_data[['constructorId', 'raceId', 'positionOrder', 'constructor_wins']].head(10))


      constructorId  raceId  positionOrder  constructor_wins
0                 1       1             20                 0
1126              1       1             19                 0
1                 1       2              7                 0
1127              1       2             20                 0
2                 1       3              6                 0
1128              1       3              5                 0
3                 1       4              4                 0
1129              1       4             12                 0
4                 1       5              9                 0
1130              1       5             16                 0


In [10]:
# Driver Exp with Constructor

# Step 1: Sort by driverId, constructorId, and raceId to ensure sequential counting
merged_data = merged_data.sort_values(by=['driverId', 'constructorId', 'raceId'])

# Step 2: Calculate cumulative experience with each constructor for each driver
merged_data['driver_exp_with_constructor'] = merged_data.groupby(['driverId', 'constructorId']).cumcount() + 1

# Display the first few rows to verify
print(merged_data[['driverId', 'constructorId', 'raceId', 'driver_exp_with_constructor']].head(10))


   driverId  constructorId  raceId  driver_exp_with_constructor
0         1              1       1                            1
1         1              1       2                            2
2         1              1       3                            3
3         1              1       4                            4
4         1              1       5                            5
5         1              1       6                            6
6         1              1       7                            7
7         1              1       8                            8
8         1              1       9                            9
9         1              1      10                           10


In [11]:
# Step 1: Constructor Experience
# Sort by constructorId and raceId to ensure cumulative experience is calculated sequentially
merged_data = merged_data.sort_values(by=['constructorId', 'raceId'])
merged_data['constructor_experience'] = merged_data.groupby('constructorId').cumcount() + 1

# Step 2: Driver Experience
# Sort by driverId and raceId to ensure cumulative experience is calculated sequentially
merged_data = merged_data.sort_values(by=['driverId', 'raceId'])
merged_data['driver_experience'] = merged_data.groupby('driverId').cumcount() + 1

# Step 3: Driver Age
# Assuming `dob` is in the `drivers` dataset and `date` is in the `races` dataset, we merge these
# if needed to calculate driver age at the time of each race
# Convert date columns if needed
if 'dob' in merged_data.columns and 'year' in merged_data.columns:
    # Calculate age (approximate years)
    merged_data['driver_age'] = (merged_data['year'] - pd.to_datetime(merged_data['dob']).dt.year)

# Step 4: Driver Wins
# Create a binary column indicating if the driver won the race (positionOrder == 1)
merged_data['win'] = (merged_data['positionOrder'] == 1).astype(int)

# Calculate cumulative wins for each driver
merged_data['driver_wins'] = merged_data.groupby('driverId')['win'].cumsum()

# Step 5: Constructor Wins
# Create a binary column indicating if the constructor won the race (positionOrder == 1)
merged_data['constructor_win'] = (merged_data['positionOrder'] == 1).astype(int)

# Calculate cumulative wins for each constructor
merged_data['constructor_wins'] = merged_data.groupby('constructorId')['constructor_win'].cumsum()

# Step 6: Driver Experience with Constructor
# Sort by driverId, constructorId, and raceId to ensure sequential counting
merged_data = merged_data.sort_values(by=['driverId', 'constructorId', 'raceId'])
merged_data['driver_exp_with_constructor'] = merged_data.groupby(['driverId', 'constructorId']).cumcount() + 1

# Drop temporary columns used for wins
merged_data.drop(columns=['win', 'constructor_win'], inplace=True)

# Display the final table with all feature-engineered variables
merged_data.head()


,resultId,raceId,driverId,constructorId,number,grid,position_driver,positionText,positionOrder,points_driver,...,position_qualifying,podium,constructor_experience,driver_experience,dob,date,driver_age,driver_wins,constructor_wins,driver_exp_with_constructor
0,7573,1,1,1,1,18,\N,D,20,0.0,...,15.0,0,1,1,1985-01-07,2009-03-29,NaN,0,0,1
1,7580,2,1,1,1,12,7,7,7,1.0,...,13.0,0,3,2,1985-01-07,2009-04-05,NaN,0,0,2
2,7599,3,1,1,1,9,6,6,6,3.0,...,9.0,0,5,3,1985-01-07,2009-04-19,NaN,0,0,3
3,7617,4,1,1,1,5,4,4,4,5.0,...,5.0,0,7,4,1985-01-07,2009-04-26,NaN,0,0,4
4,7642,5,1,1,1,14,9,9,9,0.0,...,14.0,0,9,5,1985-01-07,2009-05-10,NaN,0,0,5


In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Step 1: Define Features and Target Variable
features = merged_data[['grid', 'points_driver', 'points_standings', 'constructor_experience', 
                        'driver_experience','driver_age', 'driver_wins', 'constructor_wins', 
                        'driver_exp_with_constructor']]
target = merged_data['podium']

# Step 2: Data Preprocessing
# Replace non-numeric placeholders ('\\N') with NaN
features = features.replace('\\N', np.nan)

# Convert columns to numeric to ensure all are suitable for imputation
features = features.apply(pd.to_numeric, errors='coerce')

# Handle missing values
imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

# Scale the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_imputed)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Step 3: Model Training and Evaluation
# Define the models
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVC': SVC(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

# Train each model and evaluate performance
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    print(f"\n{model_name} Performance:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))



Random Forest Performance:
Accuracy: 0.9911387631975868
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4650
           1       0.96      0.97      0.96       654

    accuracy                           0.99      5304
   macro avg       0.98      0.98      0.98      5304
weighted avg       0.99      0.99      0.99      5304


SVC Performance:
Accuracy: 0.9711538461538461
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4650
           1       0.90      0.86      0.88       654

    accuracy                           0.97      5304
   macro avg       0.94      0.92      0.93      5304
weighted avg       0.97      0.97      0.97      5304


Logistic Regression Performance:
Accuracy: 0.9374057315233786
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      4650
           1       0.79      0.67      0.72       654

    accuracy   

Random Forest:
Accuracy: 99.1%
High precision and recall for both classes, indicating it’s effective at identifying podium finishes with minimal misclassification.
Strongest performance among the models, with balanced precision, recall, and F1-score.


Support Vector Classifier (SVC):
Accuracy: 97.1%
High precision and recall for the non-podium class (0) but slightly lower recall for the podium class (1), which indicates some difficulty in detecting all podium instances.
Still an effective model, but slightly less accurate than Random Forest, particularly in handling the minority class.


Logistic Regression:
Accuracy: 93.7%
Lower recall for the podium class, indicating that it has more difficulty identifying podium finishes.
This model is the least accurate overall, suggesting that the relationships in the data may be better captured by non-linear models like Random Forest and SVC.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np

# Assuming 'merged_data' is preloaded with all required engineered features
# Define the features and target
features = merged_data[['driverId', 'grid', 'circuitId', 'constructor_experience', 
                        'driver_experience', 'driver_age', 'driver_wins', 
                        'constructor_wins', 'driver_exp_with_constructor']]
target = merged_data['podium']

# Step 1: Data Preprocessing
# Handle missing values
imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

# Scale the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_imputed)

# Train the Random Forest model
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(features_scaled, target)

# Step 2: Create Prediction Function
def predict_podium(driver_name, grid_position, circuit_name):
    # Find driverId based on driver name
    driver_id = merged_data[merged_data['Name'] == driver_name]['driverId'].iloc[0]
    
    # Find circuitId based on circuit name
    circuit_id = merged_data[merged_data['name_x'] == circuit_name]['circuitId'].iloc[0]
    
    # Retrieve the most recent constructor experience and driver experience with constructor values
    latest_data = merged_data[(merged_data['driverId'] == driver_id) & (merged_data['circuitId'] == circuit_id)].iloc[-1]
    
    # Prepare the input data
    input_data = pd.DataFrame({
        'driverId': [driver_id],
        'grid': [grid_position],
        'circuitId': [circuit_id],
        'constructor_experience': [latest_data['constructor_experience']],
        'driver_experience': [latest_data['driver_experience']],
        'driver_age': [latest_data['driver_age']],
        'driver_wins': [latest_data['driver_wins']],
        'constructor_wins': [latest_data['constructor_wins']],
        'driver_exp_with_constructor': [latest_data['driver_exp_with_constructor']]
    })

    # Process input data (Imputation and Scaling)
    input_data_imputed = imputer.transform(input_data)
    input_data_scaled = scaler.transform(input_data_imputed)
    
    # Predict podium probability
    podium_probability = rf_model.predict_proba(input_data_scaled)[0][1] * 100
    
    return f"The probability of {driver_name} achieving a podium finish at {circuit_name} from grid position {grid_position} is {podium_probability:.2f}%."


In [29]:
print(predict_podium(driver_name="Charles Leclerc", grid_position=14, circuit_name="Baku City Circuit"))


The probability of Charles Leclerc achieving a podium finish at Baku City Circuit from grid position 14 is 0.00%.
